# MNE-Python Basic Preprocess

In [121]:
import pyxdf
import mne
import numpy as np
streams, header = pyxdf.load_xdf("sub-P301\ses-S001\eeg\sub-P301_ses-S001_task-Default_run-001_eeg.xdf") #Example Data from Lab Recoder

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\STUDENT\AppData\Local\Temp\ipykernel_10564\1362589199.py:4: SyntaxWarning: invalid escape sequence '\s'
  streams, header = pyxdf.load_xdf("sub-P301\ses-S001\eeg\sub-P301_ses-S001_task-Default_run-001_eeg.xdf") #Example Data from Lab Recoder
Stream 3: Calculated effective sampling rate 54.3723 Hz is different from specified rate 250.0000 Hz.


In [122]:
streams

[{'info': defaultdict(list,
              {'name': ['EMG_Rawtime'],
               'type': ['EMG'],
               'channel_count': ['8'],
               'nominal_srate': ['250'],
               'channel_format': ['float32'],
               'source_id': ['openbcigui'],
               'version': ['1.1000000000000001'],
               'created_at': ['2962.9573205000002'],
               'uid': ['fa9747e3-49ba-44ec-8693-8fca5ef182e7'],
               'session_id': ['default'],
               'hostname': ['STUDENT494'],
               'v4address': [None],
               'v4data_port': ['16573'],
               'v4service_port': ['16573'],
               'v6address': [None],
               'v6data_port': ['16573'],
               'v6service_port': ['16573'],
               'desc': [None],
               'stream_id': 2,
               'effective_srate': np.float64(250.27573278520182)}),
  'footer': {'info': defaultdict(list,
               {'first_timestamp': ['4265.520306'],
               

In [123]:
raw_data = streams[0]["time_series"].T #From Steam variable this query is EMG data

In [124]:
raw_data.shape # It contain with 8 channel of EMG data

(8, 43060)

In [125]:
channels = ['CH1','CH2','CH3','CH4','CH5','CH6','CH7','CH8'] #Set your target EMG channel name
info = mne.create_info(
    ch_names= channels,
    ch_types= ['emg']*len(channels),
    sfreq= 250 #OpenBCI Frequency acquistion
)
# Create MNE rawarray
raw_mne = mne.io.RawArray(raw_data, info, verbose=False)

In [126]:
raw_mne

<RawArray | 8 x 43060 (172.2 s), ~2.6 MB, data loaded>

# Create MNE Epoch

In [127]:
event_index = streams[2]["time_series"].T[0] #Get all event marker of experiment 
event_timestamp = streams[2]["time_stamps"].T #Timestamp when event marked

In [128]:
event_index

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [129]:
event_timestamp

array([4265.50549061, 4265.5094862 , 4265.5134818 , ..., 4437.54375676,
       4437.54775235, 4437.55174794])

In [130]:
events = np.column_stack((np.array(event_timestamp, dtype = int),
                        np.zeros(len(event_timestamp), dtype = int),
                        np.array(event_index, dtype = int)))

events_id = { # Set up your event name
     'Maker point': 1,
}

mne_epochs = mne.Epochs(raw_mne, events, 
        tmin= -0.5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax= 1.5,    # final timestamp (10 means set epoch duration 10 second)
        event_id =events_id,
        preload = True,
        baseline=(-0.5, 0)
    )

Not setting metadata
19 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 19 events and 501 original time points ...
0 bad epochs dropped


In [131]:
mne_epochs

<Epochs | 19 events (all good), -0.5 – 1.5 s (baseline -0.5 – 0 s), ~606 kB, data loaded,
 'Maker point': 19>

In [132]:
mne_epochs.plot(scalings=200, picks="Maker point")  # เลือกเฉพาะ EEG channels
mne_epochs.filter(
    l_freq=20.0, 
    h_freq=120.0, 
    method='iir', 
    iir_params={"order": 6, "ftype": 'butter'},
    picks="Maker point"  # เลือกเฉพาะ EEG channels
)

ValueError: picks ('Maker point') could not be interpreted as channel names (no channel "[np.str_('Maker point')]"), channel types (no type "Maker point" present), or a generic type (just "all" or "data")